In [1]:
%pip install pandas morfessor 


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
#unsupervised

#TODO: Add command line script for training the model

import pandas as pd
import morfessor

input_tsv = 'data\\eng.sentence.test.gold.tsv'  #path to the input tsv file
output_tsv = 'outputs\\eng.sentence.test.morfessor_guess.tsv'  #path tot the output tsv file
bi_model = 'models\\m_morfessor.bin'

io = morfessor.MorfessorIO()
model = io.read_binary_model_file(bi_model)



df_guess = pd.read_csv(input_tsv, sep='\t', header=None)
data_guess = df_guess[0].astype(str)
i = 0

#segment on words

for entry in data_guess:
  sent = ''
  j = 0
  words = entry.split() #list of words
  length_sent = len(words)  #number of words

  for word in words:
    list_word = model.viterbi_segment(word)[0]
    k = 0
    length_word = len(list_word)  #number of morphs

    for morph in list_word:
      if k != length_word - 1:  #not last morph
        sent += (morph + ' @@')
        k+=1
      else: #last morph
        if j != length_sent - 1:
          sent += (morph + ' ')
        else :
          sent +=morph
    j+=1

  df_guess[1][i] = sent
  i+=1

df_guess.to_csv(output_tsv, sep='\t', header=None, index = False)

C:\Users\admin\AppData\Local\Temp\ipykernel_2636\1302120141.py:42: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df_guess[1][i] = sent
